[GitHub](https://github.com/elsanns/xai-nlp-notebooks/blob/master/electra_fine_tune_explain_captum_ig.ipynb)

# Content
---


This notebook contains an example of [fine-tuning](https://huggingface.co/transformers/training.html) an [Electra](https://huggingface.co/transformers/model_doc/electra.html) model on the [GLUE SST-2](https://nlp.stanford.edu/sentiment/index.html) dataset. After fine-tuning, the [Integrated Gradients](https://arxiv.org/pdf/1703.01365.pdf) **interpretability** method is applied to compute tokens' attributions for each target class. 
* We will instantiate a pre-trained Electra model from the [Transformers](https://huggingface.co/transformers/) library. 
* The data is downloaded from the [nlp](https://huggingface.co/nlp/) library. The input text is tokenized with [ElectraTokenizerFast](https://huggingface.co/transformers/model_doc/electra.html#electratokenizerfast) tokenizer backed by HF [tokenizers](https://huggingface.co/transformers/main_classes/tokenizer.html) library.
* **Fine-tuning** for sentiment analysis is handled by the [Trainer](https://huggingface.co/transformers/main_classes/trainer.html) class. 
* After fine-tuning, the [Integrated Gradients](https://captum.ai/api/integrated_gradients.html) interpretability algorithm will assign importance scores to
input tokens. We will use a **PyTorch** implementation from the [Captum](https://captum.ai/) library. 
  - The algorithm requires providing a reference sample (a baseline) since importance attribution is performed based on the model's output, as inputs change from reference values to the actual sample. 
  - The Integrated Gradients method satisfies the [completeness](http://theory.stanford.edu/~ataly/Talks/sri_attribution_talk_jun_2017.pdf) property. We will look at the sum of attributions for a sample and show that the sum approximates (explains) prediction's shift from the baseline value. 
* The final sections of this notebook contain a colour-coded **visualization** of attribution results made with *captum.attr.visualization* library.

The notebook is based on the [Hugging Face documentation](https://huggingface.co/) and the implementation of Integrated Gradients attribution methods is adapted from the Captum.ai
[Interpreting BERT Models (Part 1)](https://captum.ai/tutorials/Bert_SQUAD_Interpret).

# Installation & imports

---

In [ ]:
!pip install transformers
# pyarrow version as required by nlp v 0.3.0 (runtime restart in Colab)
!pip install "pyarrow==0.16.0"  
!pip install nlp
!pip install captum

In [ ]:
from typing import Dict

import matplotlib.pyplot as plt
import nlp
import numpy as np
import pandas as pd
import torch
import transformers
from captum.attr import (IntegratedGradients, LayerIntegratedGradients,
                         configure_interpretable_embedding_layer,
                         remove_interpretable_embedding_layer)
from captum.attr import visualization as viz
from torch.utils.data import Dataset
from transformers import (ElectraForSequenceClassification,
                          ElectraTokenizerFast, EvalPrediction, InputFeatures,
                          Trainer, TrainingArguments, glue_compute_metrics)

transformers.__version__

# Model

---

Sentiment analysis is a classification task that requires assigning a label to an entire sentence (sequence). We will use a PyTorch implementation of [ElectraForSequenceClassification](https://huggingface.co/transformers/model_doc/electra.html#electraforsequenceclassification) from the Hugging Face library. A matching tokenizer implemented in the [ElectraTokenizerFast](https://huggingface.co/transformers/model_doc/electra.html#electratokenizerfast) class will handle tokenization.

In [4]:
model = ElectraForSequenceClassification.from_pretrained(
    "google/electra-small-discriminator", num_labels = 2)

tokenizer = ElectraTokenizerFast.from_pretrained(
    "google/electra-small-discriminator", do_lower_case=True)                      

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

# Data

---


**Download**

Let's now download the SST-2 dataset from the nlp library and take a brief look at it. It contains short movie reviews labelled for sentiment: 0 for negative and 1 for a positive review. The data is split into training, validation and test set. The labels for the test set are kept undisclosed.

In [5]:
# Load the SST2 dataset from the nlp library
dataset = nlp.load_dataset('glue', 'sst2')

# Look at the labels
print("Training set labels: {}".format(set(dataset['train']['label'])))
print("Validation set labels: {}".format(set(dataset['validation']['label'])))
print("Test set labels: {}".format(set(dataset['test']['label'])))

# Explore the dataset
df = pd.DataFrame({"senence": dataset["train"]["sentence"],
                   "label": dataset["train"]["label"]})
pd.options.display.max_colwidth = 0
df.head()

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/005857b1e5a6280d8f1a9b9537d44a08ba30cb6be958e81fac98e625a0d487a7. Subsequent calls will reuse this data.
Training set labels: {0, 1}
Validation set labels: {0, 1}
Test set labels: {-1}


,senence,label
0,hide new secretions from the parental units,0
1,"contains no wit , only labored gags",0
2,that loves its characters and communicates something rather beautiful about human nature,1
3,remains utterly satisfied to remain the same throughout,0
4,on the worst revenge-of-the-nerds clichés the filmmakers could dredge up,0


**Create dataset**

We will now create a custom [map-style PyTorch dataset](https://pytorch.org/docs/stable/data.html#map-style-datasets) to serve model's key-value parameters in a seamless manner. 

The `TrainerDataset` class is derived from `torch.utils.data.Dataset`. The overridden `__getitem__` method yields a Python *Object* 
for compatibility with the [DefaultDataCollator](https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py), in this example, the `InputFeatures` class is used. 

Conversion to torch tensors and placing on cuda/cpu is handled by the trainer object used for fine-tuning.

In [6]:
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True)   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])         

We need to create the training and validation datasets. As GLUE SST-2 dataset does not disclose labels for the test set, we will be using validation data for testing.

In [7]:
train_dataset = TrainerDataset(dataset["train"]["sentence"],
                               dataset["train"]["label"], tokenizer)
eval_dataset = TrainerDataset(dataset["validation"]["sentence"],
                              dataset["validation"]["label"], tokenizer)

# Fine-tuning

---

Fine-tuning with a `Trainer` class instance requires setting training arguments and creating a trainer object. The model, as well as training and validation datasets, are passed to the trainer's constructor, along with training arguments. The `Trainer` class takes care of conversion to tensor format and placement on a cpu/gpu device.

## Set parameters

Training parameters have been taken from the [Electra Github](https://github.com/google-research/electra/blob/master/configure_finetuning.py) repository or are default values. 

In [8]:
# Set seed for reproducibility
np.random.seed(123)
torch.manual_seed(123)

training_args = TrainingArguments(
    output_dir="./models/model_electra",
    num_train_epochs=3,
    overwrite_output_dir=True,
    do_train=True,
    per_device_train_batch_size=32,
    dataloader_drop_last=True,  # Make sure all batches are of equal size
)


def compute_metrics(p: EvalPrediction) -> Dict:
    preds = np.argmax(p.predictions, axis=1)
    # The choice of a dataset (task_name) implies metric
    return glue_compute_metrics(
        task_name="sst-2",
        preds=preds,
        labels=p.label_ids)


# Instantiane the Trainer class
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics)

## Run fine-tuning

Run the `train` method of the trainer object to fine-tune the model on the SST-2 dataset.

In [9]:
trainer.train()

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


TrainOutput(global_step=6312, training_loss=0.18736900893911715)

## Evaluate 

The metric used for evaluation of the Stanford Sentiment Treebank (SST) data is *Accuracy*. The result is returned by the `Trainer` class object used for fine-tuning. 

In [ ]:
model_result = trainer.evaluate()
print("Accuracy: {}".format(model_result["eval_acc"]))

# Interpretability with Captum

---

The examples below use two attribution methods from the Captum library:
- **Integrated Gradients** - the method requires configuring interpretation hooks to perform attribution for all three embedding layers in one step, and
- **Layer Integrated Gradients**, computed separately with respect to each of the three layers:
    - `model.electra.embeddings.word_embeddings`
    - `model.electra.embeddings.token_type_embeddings`
    - `model.electra.embeddings.position_embeddings`

We will try to find out to what extent, according to these methods, each token has contributed to the model's prediction, or, more precisely, to its shift from the baseline output. 
Each method requires setting a target class index: 0 for negative or 1 for a positive sentiment. Attribution is performed for each target class separately. Scores will be assigned with regard to the model's output for the selected class.

The shape of attributions is the same as the shape of the *inputs* parameter of the `attribute` method.

Let's pick an example:

In [ ]:
text = "visually imaginative , thematically instructive and thoroughly \
delightful , it takes us on a roller-coaster ride from innocence to experience \
without even a hint of that typical kiddie-flick sentimentality . "
true_label = 1

[x for x in dataset["validation"] if x["sentence"] == text]

## Prepare input 

Set a cpu/gpu device according to availability. 

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

**Helper functions**


The functions below **construct input tensors** for our sample and for a sequence of [PAD] tokens serving as a baseline. We also need to define a **forward function** running inference on the model. The function will be passed on to objects handling attribution. 

Computation with **IntegratedGradients** requires altering the model by **configuring additional layers**. For this purpose, the Captum library provides the `configure_interpretable_embedding_layer` and `remove_interpretable_embedding_layer` methods. Configuring an interpretable embedding layer modifies the model. A model with interpretable layers requires input of a different shape. 

In [ ]:
def predict_forward_func(input_ids, token_type_ids=None, 
                         position_ids=None, attention_mask=None):
    """Function passed to ig constructors"""
    return model(input_ids=input_ids, 
                 token_type_ids=token_type_ids, 
                 position_ids=position_ids, 
                 attention_mask=attention_mask)[0]  


def prepare_input(text):
    """Prepare input ig attribution: tokenize sample and baseline text. """
    tokenized_text = tokenizer(text, return_tensors="pt", 
                               return_attention_mask=True)
    seq_len = tokenized_text["input_ids"].shape[1]
    position_ids = torch.arange(seq_len, dtype=torch.long).unsqueeze(0)

    # Construct the baseline (a reference sample).
    # Sequence of [PAD] tokens of length equal to that of the processed samples
    ref_text = tokenizer.pad_token * (seq_len - 2) # special tokens
    tokenized_ref_text = tokenizer(ref_text, return_tensors="pt") 
    ref_position_ids = torch.arange(seq_len, dtype=torch.long).unsqueeze(0)

    return (tokenized_text["input_ids"],
            tokenized_text["token_type_ids"], 
            position_ids,
            tokenized_ref_text["input_ids"],
            tokenized_ref_text["token_type_ids"], 
            ref_position_ids,
            tokenized_text["attention_mask"])   


def configure_interpretable_embeddings():
    """Configure interpretable embedding layer"""
    interpretable_embedding1 = \
    configure_interpretable_embedding_layer(
        model,
        'electra.embeddings.word_embeddings')
    interpretable_embedding2 = \
    configure_interpretable_embedding_layer(
        model,
        'electra.embeddings.token_type_embeddings')
    interpretable_embedding3 = \
    configure_interpretable_embedding_layer(
        model,
        'electra.embeddings.position_embeddings')
    return (interpretable_embedding1,\
            interpretable_embedding2,\
            interpretable_embedding3)


def remove_interpretable_embeddings(interpretable_embedding1, 
                                    interpretable_embedding2, 
                                    interpretable_embedding3):
    '''Remove interpretable layer to restore oryginal model structure'''
    if not type(model.get_input_embeddings()).__name__ == "InterpretableEmbeddingBase":
        return
    remove_interpretable_embedding_layer(model, interpretable_embedding1)
    remove_interpretable_embedding_layer(model, interpretable_embedding2)
    remove_interpretable_embedding_layer(model, interpretable_embedding3)    


def prepare_input_embed(input_ids, token_type_ids, position_ids,
                        ref_input_ids, ref_token_type_ids, ref_position_ids,
                        attention_mask):
    """Construct input for the modified model"""
    input_ids_embed = interpretable_embedding1.indices_to_embeddings(input_ids)
    ref_input_ids_embed = interpretable_embedding1.indices_to_embeddings(
        ref_input_ids)
    token_type_ids_embed = interpretable_embedding2.indices_to_embeddings(
        token_type_ids)
    ref_token_type_ids_embed = interpretable_embedding2.indices_to_embeddings(
        ref_token_type_ids)
    position_ids_embed = interpretable_embedding3.indices_to_embeddings(
        position_ids)
    ref_position_ids_embed = interpretable_embedding3.indices_to_embeddings(
        ref_position_ids)
    
    return (input_ids_embed, token_type_ids_embed, position_ids_embed,\
    ref_input_ids_embed, ref_token_type_ids_embed, ref_position_ids_embed, \
    attention_mask)


def place_on_device(*tensors):
    tensors_device = []
    for t in tensors:
        tensors_device.append(t.to(device))
    return tuple(tensors_device)  

## Integrated Gradients

To compute attributions with Integrated Gradients we will:
- instantiate the `IntegratedGradients` class passing the `predict_forward_func` function as parameter,
- configure interpretable embeddings layer,
- prepare input tensors,
- compute attributions,
- remove interpratable embeddings layer.

### Compute attributions

In [ ]:
# Instantiate the IntegratedGradients class
ig = IntegratedGradients(predict_forward_func)

In [ ]:
# Configure interpretable embeddings layer 
print("Oryginal model input embeddings:\n {}\n".
      format(model.get_input_embeddings()))
if not type(model.get_input_embeddings()).__name__ == "InterpretableEmbeddingBase":
    interpretable_embedding1, interpretable_embedding2, interpretable_embedding3 =\
    configure_interpretable_embeddings()
print("Input embeddings with interpretable layer:\n {}\n".
      format(model.get_input_embeddings()))

# Prepare input 
input_data = prepare_input(text)
input_data = place_on_device(*input_data) 
input_data_embed = prepare_input_embed(*input_data) 
input_ids_embed, token_type_ids_embed, position_ids_embed = input_data_embed[0:3]
ref_input_ids_embed, ref_token_type_ids_embed, \
ref_position_ids_embed = input_data_embed[3:6]
attention_mask = input_data_embed[-1]

# Compute attributions for both target classes
# class 0 (negative)
attributions_0, approximation_error_0 = ig.attribute(
    inputs=(input_ids_embed, token_type_ids_embed, position_ids_embed),
    baselines=(ref_input_ids_embed, 
               ref_token_type_ids_embed, 
               ref_position_ids_embed),
               additional_forward_args=(attention_mask),
               target = 0, # Set target class here
               return_convergence_delta=True, 
               n_steps=200)
# class 1 (positive)
attributions_1, approximation_error_1 = ig.attribute(
    inputs=(input_ids_embed, token_type_ids_embed, position_ids_embed),
    baselines=(ref_input_ids_embed, 
               ref_token_type_ids_embed, 
               ref_position_ids_embed),
               additional_forward_args=(attention_mask),
               target = 1, # Set target class here
               return_convergence_delta=True, 
               n_steps=200)

# Remove interpratable embeddings layer used by ig attribution
remove_interpretable_embeddings(interpretable_embedding1, 
                                interpretable_embedding2, 
                                interpretable_embedding3)
print("\nInput embeddings with interpretable layer removed:\n {}\n"
.format(model.get_input_embeddings()))

### Completeness

The Integrated Gradients method satisfies the completeness property. The sum of attributions should be equal, with certain accuracy, to the difference between the model's output for the sample and its output for the selected baseline (in this case a sequence of [PAD] tokens). Increase the parameter `n_steps` parameter of the `ig.attribute` method to obtain better accuracy.

In [ ]:
def check_completeness(attributions_0, attributions_1):
    input_ids, token_type_ids, position_ids, \
    ref_input_ids, ref_token_type_ids, ref_position_ids, attention_mask = input_data

    # Prediction for the sample
    scores = predict_forward_func(input_ids, token_type_ids,
                                position_ids, attention_mask) 

    # Prediction for the baseline
    ref_scores = predict_forward_func(ref_input_ids, ref_token_type_ids,
                                    ref_position_ids, attention_mask)

    # How prediction for the sample differs from baseline prediction  
    diff_from_baseline = scores - ref_scores
    diff_from_baseline = diff_from_baseline.clone().detach().to('cpu').numpy()[0]

    # Put on cpu
    if torch.is_tensor(attributions_0[0]):
        attributions_0 = [x.clone().detach().to('cpu').numpy() for x in attributions_0]
    if torch.is_tensor(attributions_1[0]):
        attributions_1 = [x.clone().detach().to('cpu').numpy() for x in attributions_1]

    # Sum of attributions
    attributions_sum0 = [x.sum() for x in attributions_0]
    attributions_sum1 = [x.sum() for x in attributions_1]
    attributions_sum = [sum(attributions_sum0), sum(attributions_sum1)]
    diff = diff_from_baseline - attributions_sum

    # Find out which layers contribute to the score (order: order of inputs)
    print("Class 0: input_ids sum: {}".format(attributions_sum0[0]))
    print("Classs 0: token_type sum: {}".format(attributions_0[1].sum()))
    print("Class 0: position_ids sum: {}".format(attributions_0[2].sum()))
    print("Class 1: input_ids sum: {}".format(attributions_1[0].sum()))
    print("Classs 1: token_type sum: {}".format(attributions_1[1].sum()))
    print("Class 1: position_ids sum: {}".format(attributions_1[2].sum()))

    # Compare sum of attributions and baseline prediction - prediction
    print("\nPrediction for sample: {}".format(scores))
    print("Prediction for baseline: {}".format(ref_scores))
    print("Difference from baseline: {}".format(diff_from_baseline))
    print("Sum of attributions: {}".format(attributions_sum))
    print("\nClass 0:\n score: {}\n reference score: {}\
    \n difference from ref.:{}\n attributions: {}\
    \n difference from reference - attributions: {}".\
    format(scores[0][0], ref_scores[0][0], diff_from_baseline[0], 
            attributions_sum[0], diff[0]))
    print("\nClass 1:\n score: {}\n reference score: {}\
    \n difference from ref.:{}\n attributions: {}\
    \n difference from reference - attributions: {}".\
    format(scores[0][1], ref_scores[0][1], diff_from_baseline[1], 
            attributions_sum[1], diff[1]))
    
    return attributions_0, attributions_1
    
    
attributions_0, attributions_1 = check_completeness(attributions_0, 
                                                    attributions_1)    

## Layer Integrated Gradients

With Layer Integrated Gradients, attributions are computed with respect to a certain layer. We'll run the algorithm for three [layers](https://github.com/huggingface/transformers/blob/d5b0a0e235cc6fccba4f9013cdb54cee01e90a91/src/transformers/modeling_electra.py#L131) separately: 
- `model.electra.embeddings.word_embeddings`
- `model.electra.embeddings.token_type_embeddings`
- `model.electra.embeddings.position_embeddings`



### Compute attributions

Assigning attributions with Layer Integrated Gradients requires:
- instantiating the `IntegratedGradients` class and passing the `predict_forward_func` function along with a selected layer as parameters,
- calling `lig.attribute` to assign values to each token.

In [ ]:
# 1. model.electra.embeddings.word_embeddings
lig_word_we = LayerIntegratedGradients(
    predict_forward_func, 
    model.electra.embeddings.word_embeddings)

layer_attributions_we_0, _ = lig_word_we.attribute(
    inputs=input_ids, baselines=ref_input_ids,
    additional_forward_args=(token_type_ids, position_ids, attention_mask),
    return_convergence_delta=True, target=0, n_steps=200)

layer_attributions_we_1, _ = lig_word_we.attribute(
    inputs=input_ids, baselines=ref_input_ids,
    additional_forward_args=(token_type_ids, position_ids, attention_mask),
    return_convergence_delta=True, target=1, n_steps=200)

# 2. model.electra.embeddings.token_type_embeddings
lig_word_tte = LayerIntegratedGradients(
    predict_forward_func, 
    model.electra.embeddings.token_type_embeddings)

layer_attributions_tte_0, _ = lig_word_tte.attribute(
    inputs=input_ids, baselines=ref_input_ids,
    additional_forward_args=(token_type_ids, position_ids, attention_mask),
    return_convergence_delta=True, target=0, n_steps=200)

layer_attributions_tte_1, _ = lig_word_tte.attribute(
    inputs=input_ids, baselines=ref_input_ids,
    additional_forward_args=(token_type_ids, position_ids, attention_mask),
    return_convergence_delta=True, target=1, n_steps=200)

# 3. model.electra.embeddings.position_embeddings
lig_word_pe = LayerIntegratedGradients(
    predict_forward_func, 
    model.electra.embeddings.position_embeddings)

layer_attributions_pe_0, _ = lig_word_pe.attribute(
    inputs=input_ids, baselines=ref_input_ids,
    additional_forward_args=(token_type_ids, position_ids, attention_mask),
    return_convergence_delta=True, target=0, n_steps=200)

layer_attributions_pe_1, _ = lig_word_pe.attribute(
    inputs=input_ids, baselines=ref_input_ids,
    additional_forward_args=(token_type_ids, position_ids, attention_mask),
    return_convergence_delta=True, target=1, n_steps=200)

**Plot**

Attributions assigned to tokens may take opposite values when computed with regard to class 0 and class 1.

In [ ]:
# word_embeddings: index 0
lig_0 = layer_attributions_0[0].squeeze().sum(1)
lig_1 = layer_attributions_1[0].squeeze().sum(1)

tokens = tokenizer.convert_ids_to_tokens(tokenizer(text)["input_ids"])

plt.rcParams["figure.figsize"] = [12, 6]
plt.bar(list(range(len(lig_0))), lig_0, color='r', alpha=0.5)
plt.bar(list(range(len(lig_1))), lig_1, color='g', alpha=0.5)
plt.xticks(list(range(len(lig_0))), tokens, rotation='vertical')
plt.legend(labels=["Target: negative", "Target: positive"])
plt.xlabel('Token', fontweight='bold')
plt.title("Token attributions for positive and negative target class")
plt.show()

### Completeness

Completeness for attributions found for each layer separately

In [ ]:
layer_attributions_0, layer_attributions_1 = check_completeness(
    (layer_attributions_we_0, layer_attributions_tte_0, layer_attributions_pe_0),
    (layer_attributions_we_1, layer_attributions_tte_1, layer_attributions_pe_1)
)

### Compare with IG

In [ ]:
# Attributions for input_ids
ig_1 = attributions_1[0].squeeze().sum(1)
lig_1 = layer_attributions_1[0].squeeze().sum(1)

range_ig = [x + 0.5 for x in np.arange(len(ig_1))]
range_lig = [x + 0.5 for x in range_ig]
 
plt.bar(range_ig, ig_1, width=0.5, label='ig')
plt.bar(range_lig, lig_1, width=0.5, label='lig')
plt.xlabel('Token', fontweight='bold')
plt.xticks(list(range(len(lig_1))), tokens, rotation='vertical')
plt.legend()
plt.title("Attributions with IG and LIG for the positive target class.")
plt.show()

## Visualization

### Helper functions

In [ ]:
def get_input_data(text):
    input_data = place_on_device(*prepare_input(text))
    input_data_embed = prepare_input_embed(*input_data)   
    return input_data, input_data_embed 


def ig_attribute(class_index, input_data_embed):
    return ig.attribute(inputs=input_data_embed[0:3],
                        baselines=input_data_embed[3:6],
                        additional_forward_args=(input_data_embed[6]),
                        target = class_index,
                        return_convergence_delta=True,
                        n_steps=200)
    

def lig_attribute(class_index, input_data):
    return lig.attribute(
        inputs=input_data[0], baselines=input_data[3],
        additional_forward_args=(input_data[1], input_data[2], input_data[6]),
        return_convergence_delta=True, target=class_index)


def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions


def compute_attributions_ig(input_data_embed):
    # Create interpretable layer
    if not type(
        model.get_input_embeddings()).__name__ == "InterpretableEmbeddingBase":    
        interpretable_embedding1, interpretable_embedding2,\
        interpretable_embedding3 = configure_interpretable_embeddings()
    # Compute attributions for positive and nagative samples (class 1 and 0)
    attr_0, delta_0 = ig_attribute(0, input_data_embed)
    attr_1, delta_1 = ig_attribute(1, input_data_embed)
    # Remove interprateble layer used by ig attribution
    remove_interpretable_embeddings(interpretable_embedding1, 
                                    interpretable_embedding2, 
                                    interpretable_embedding3)
    # Return attributions for 'input_ids' (element 0) for 0 and 1 class indices
    return (attr_0[0], delta_0), (attr_1[0], delta_1)    


def compute_attributions_lig(input_data):  
    # Compute attributions for positive and nagative samples (class 1 and 0)
    return lig_attribute(0, input_data), lig_attribute(1, input_data)


def get_visualization_record(text, attributions, scores, true_label,
                             all_tokens, approximation_error):
    attributions_sum = summarize_attributions(attributions)
    return viz.VisualizationDataRecord(
        attributions_sum,
        torch.max(torch.softmax(scores[0], dim=0)),
        torch.argmax(scores),
        true_label,
        text,
        attributions_sum.sum(),
        all_tokens,
        approximation_error)
    

def visualize_attributions(text, true_label, method):
    # Prepare input
    input_data = place_on_device(*prepare_input(text))
    input_data_embed = prepare_input_embed(*input_data)
    # Compute attributions
    attr_0, attr_1, delta_0, delta_1 = None, None, None, None
    if method == "ig":
        (attr_0, delta_0), (attr_1, delta_1) = \
        compute_attributions_ig(input_data_embed)
    elif method == "lig":    
        (attr_0, delta_0), (attr_1, delta_1) = \
        compute_attributions_lig(input_data)
    else:
        return "method: ig or lig"    
    # Run inference
    scores = predict_forward_func(*input_data[0:3], input_data[-1])
    # Prepare visualization 
    indices = input_data[0][0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    data_vis_0 = get_visualization_record(text, attr_0, scores, 
                                          true_label, all_tokens, delta_0)  
    data_vis_1 = get_visualization_record(text, attr_1, scores, 
                                          true_label, all_tokens, delta_1) 
    # Visualize
    print("\nAttribution method: {},".
          format(method), "class index: 0 (negative)")
    viz.visualize_text([data_vis_0])
    print("Attribution method: {},".
          format(method), "Class index: 1 (positive)")
    viz.visualize_text([data_vis_1]) 
    return attr_0, attr_1    

### Examples

Captum visualization library shows in green tokens that push the prediction towards the target class. Those driving the score towards the reference value are marked in red. As a result, words perceived as positive will appear in green if attribution is performed against class 1 (positive) but will be highlighted in red with an attribution targeting class 0 (negative).

Because importance scores ar assigned to tokens, not words, some examples may show that attribution is highly dependent on tokenization. Classification result may vary between runs.


Browse examples:

In [ ]:
# Run predictions
eval_pred_result = trainer.predict(eval_dataset)
predictions = np.argmax(eval_pred_result.predictions, axis=1)

# Find misclassifed samples
eval_samples = [tokenizer.decode(x.input_ids, skip_special_tokens=True) \
                for x in eval_dataset]
eval_preds = list(zip(eval_pred_result.label_ids, predictions))
positive_pred_as_positive = [sample for sample, (real_label, pred_label) \
                             in zip(eval_samples, eval_preds) \
                             if real_label == pred_label and real_label == 1]  
negative_pred_as_negative = [sample for sample, (real_label, pred_label) \
                             in zip(eval_samples, eval_preds) \
                             if real_label == pred_label and real_label == 0]                               
negative_pred_as_positive = [sample for sample, (real_label, pred_label) \
                             in zip(eval_samples, eval_preds) \
                             if real_label != pred_label and real_label == 0]
positive_pred_as_negative = [sample for sample, (real_label, pred_label) \
                             in zip(eval_samples, eval_preds) \
                             if real_label != pred_label and real_label == 1]

# Browse
# print('\n'.join(positive_pred_as_positive))   
# print('\n'.join(negative_pred_as_negative))    
# print('\n'.join(negative_pred_as_positive))     
# print('\n'.join(positive_pred_as_negative))                                          

#### Positive

A correctly classified positive sample

Use our example or pick your own by setting `text_vis` and `true_label_vis` variables.

In [ ]:
text_vis = text
true_label_vis = true_label

ig_0, ig_1 = visualize_attributions(text_vis, true_label_vis, "ig")
# lig_0, lig_1 = visualize_attributions(text_vis, true_label_vis, "lig")

#### Negative

A correctly classified negative sample

Use our example or pick your own by setting `text_vis` and `true_label_vis` variables.

In [ ]:
text_vis = 'the film makes a fatal mistake : it asks us to care about a young \
man whose only apparent virtue is that he is not quite as unpleasant as some \
of the people in his life.'
true_label_vis = 0

ig_0, ig_1 = visualize_attributions(text_vis, true_label_vis, "ig")
# lig_0, lig_1 = visualize_attributions(text_vis, true_label_vis, "lig")

#### Misclassified

Negative examples misclassified as positive

In [ ]:
print('\n'.join(negative_pred_as_positive)) 

Pick an example

In [ ]:
#text_vis = "..."
#true_label_vis = 0

#ig_0, ig_1 = visualize_attributions(text_vis, true_label_vis, "ig")
#lig_0, lig_1 = visualize_attributions(text_vis, true_label_vis, "lig")

Positive examples misclassified as negative

In [ ]:
print('\n'.join(positive_pred_as_negative)) 

Pick an example

In [ ]:
#text_vis = "..."
#true_label_vis = 1

#ig_0, ig_1 = visualize_attributions(text_vis, true_label_vis, "ig")
# lig_0, lig_1 = visualize_attributions(text_vis, true_label_vis, "lig")

# References

---

https://captum.ai/tutorials/Bert_SQUAD_Interpret

https://captum.ai/docs/algorithms

https://captum.ai/api/integrated_gradients.html

https://github.com/google-research/electra/

https://github.com/google-research/electra/blob/master/configure_finetuning.py

https://github.com/huggingface/nlp/blob/master/notebooks/Overview.ipynb

https://huggingface.co/transformers/main_classes/trainer.html